In [145]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',800)

In [235]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [21]:
train.shape

(4209, 378)

In [22]:
# remove the outliers
train = train.loc[train['y'] < 250]
train.shape

(4208, 378)

In [23]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

### remove zero variance features

In [24]:
# count unique counts per column
p1 = train.apply(lambda x: len(x.unique()))
p1 = p1[p1 == 1]

In [25]:
cols_to_remove = []
for k,v in p1.iteritems():
    cols_to_remove.append(k)
    

In [26]:
train.drop(cols_to_remove, axis=1, inplace=True)
test.drop(cols_to_remove, axis=1, inplace=True)
print ("We have removed {} columns from train and test data respectively".format(len(cols_to_remove)))

We have removed 12 columns from train and test data respectively


### The strategy moving forward would be to:

1. apply dimensionality reduction techniques only on binary variables but add ID variable - done
2. Create a feature is_duplicate in both files - done
3. apply logistic pca - done
4. mean of X0 - done
5. category embedding - not done
6. distance features in binary columns - done
7. distance features in categorical columns after label encoding  - not doing
8. TSNE (in categorical and binary columns) - done
9. Feature interactions (two_way, three_way, four_way) - done
10. Number of non-zero elements in each row - done

In [27]:
binary_cols = [c for c in train.columns if train[c].dtype == 'int']

We'll keep ID columns in decomposition in next round, for now let's remove the ID and check score<br />
**Round 1** : Without ID <br />
**Round 2** : Pending..

In [28]:
binary_cols.remove('ID')

In [29]:
import xgboost as xgb
import lightgbm as lgb

from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.cluster import FeatureAgglomeration

In [30]:
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.loc[:,binary_cols])
tsvd_results_test = tsvd.transform(test.loc[:,binary_cols])

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train  = pca.fit_transform(train.loc[:,binary_cols])
pca2_results_test = pca.transform(test.loc[:,binary_cols])

# ICA
ica = FastICA(n_components=n_comp, random_state=420,max_iter=1000,tol=1e-5)
ica2_results_train = ica.fit_transform(train.loc[:,binary_cols])
ica2_results_test = ica.transform(test.loc[:,binary_cols])

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.loc[:,binary_cols])
grp_results_test = grp.transform(test.loc[:,binary_cols])

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train.loc[:,binary_cols])
srp_results_test = srp.transform(test.loc[:,binary_cols])

# NMF
nmf = NMF(n_components=n_comp, init='nndsvdar', random_state=420)
nmf_results_train = nmf.fit_transform(train.loc[:,binary_cols])
nmf_results_test = nmf.transform(test.loc[:,binary_cols])

# FAG
fag = FeatureAgglomeration(n_clusters=n_comp, linkage='ward')
fag_results_train = fag.fit_transform(train.loc[:,binary_cols])
fag_results_test = fag.transform(test.loc[:,binary_cols])

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [32]:
# dim_reds = list()
# train_pca = pd.DataFrame()
# test_pca = pd.DataFrame()

# train_ica = pd.DataFrame()
# test_ica = pd.DataFrame()

# train_tsvd = pd.DataFrame()
# test_tsvd = pd.DataFrame()

# train_grp = pd.DataFrame()
# test_grp = pd.DataFrame()

# train_srp = pd.DataFrame()
# test_srp = pd.DataFrame()

# train_nmf = pd.DataFrame()
# test_nmf = pd.DataFrame()

# train_fag = pd.DataFrame()
# test_fag = pd.DataFrame()

dim_train = pd.DataFrame()
dim_test = pd.DataFrame()

for i in range(1, n_comp + 1):
    dim_train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    dim_test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    dim_train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    dim_test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    dim_train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    dim_test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    dim_train['grp_' + str(i)] = grp_results_train[:, i - 1]
    dim_test['grp_' + str(i)] = grp_results_test[:, i - 1]

    dim_train['srp_' + str(i)] = srp_results_train[:, i - 1]
    dim_test['srp_' + str(i)] = srp_results_test[:, i - 1]
    
    dim_train['nmf_' + str(i)] = nmf_results_train[:, i - 1]
    dim_test['nmf_' + str(i)] = nmf_results_test[:, i - 1]
    
    dim_train['fag_' + str(i)] = fag_results_train[:, i - 1]
    dim_test['fag_' + str(i)] = fag_results_test[:, i - 1]

In [33]:
dim_train.head()

,pca_1,ica_1,tsvd_1,grp_1,srp_1,nmf_1,fag_1,pca_2,ica_2,tsvd_2,grp_2,srp_2,nmf_2,fag_2,pca_3,ica_3,tsvd_3,grp_3,srp_3,nmf_3,fag_3,pca_4,ica_4,tsvd_4,grp_4,srp_4,nmf_4,fag_4,pca_5,ica_5,tsvd_5,grp_5,srp_5,nmf_5,fag_5,pca_6,ica_6,tsvd_6,grp_6,srp_6,nmf_6,fag_6,pca_7,ica_7,tsvd_7,grp_7,srp_7,nmf_7,fag_7,pca_8,ica_8,tsvd_8,grp_8,srp_8,nmf_8,fag_8,pca_9,ica_9,tsvd_9,grp_9,srp_9,nmf_9,fag_9,pca_10,ica_10,tsvd_10,grp_10,srp_10,nmf_10,fag_10,pca_11,ica_11,tsvd_11,grp_11,srp_11,nmf_11,fag_11,pca_12,ica_12,tsvd_12,grp_12,srp_12,nmf_12,fag_12
0,0.748303,0.009011,3.850594,1.789866,-3.761778,0.027661,0.166667,2.225544,-0.027798,0.557604,2.981657,0.000000,0.067593,0.555556,1.069468,-0.030271,2.271649,-0.384209,0.000000,0.106174,0.061224,0.929598,-0.031851,0.991541,1.175759,3.761778,0.0,0.461538,1.396531,-0.004755,-1.061111,2.504651,-1.253926,0.000000,0.1875,0.045094,0.015159,0.320606,-1.772131,0.000000,0.232460,0.0,0.606772,-0.003497,0.040414,-0.329599,1.253926,0.088011,0.6,-0.924261,-0.011807,0.186636,-0.233858,-2.507852,0.00000,0.75,0.156522,0.007042,0.443277,-1.892650,1.253926,0.014600,0.454545,-0.730783,0.000954,0.279590,1.967263,-1.253926,0.000000,0.000000,-1.777898,0.013602,-0.717413,2.697739,1.253926,0.000000,0.2,-0.639547,0.006862,1.354701,-4.562294,2.507852,0.000000,0.473684
1,-0.215018,0.010906,5.889622,-1.305572,-1.253926,0.069824,0.000000,1.123640,-0.006906,-0.203642,4.213747,0.000000,0.000000,0.444444,-0.804340,0.000652,0.960084,-0.894137,0.000000,0.097642,0.008163,-0.668803,0.007064,-1.141209,-0.741385,2.507852,0.0,0.769231,0.240267,-0.001322,-0.459641,-1.531423,-2.507852,0.054425,0.1250,0.039675,-0.020655,-0.365109,-2.007056,0.000000,0.000000,0.0,1.206874,0.015302,0.027985,0.195689,0.000000,0.104267,0.6,-0.552963,-0.013134,1.056100,0.312482,1.253926,0.15394,1.00,-0.093143,-0.012828,0.825847,-2.943465,1.253926,0.047130,0.363636,0.464558,-0.000496,-0.205890,1.365388,1.253926,0.048782,0.000000,0.092894,0.002970,0.459868,0.804004,0.000000,0.000000,0.8,1.107224,0.017836,0.593247,-4.404966,3.761778,0.191333,1.000000
2,-0.889926,0.002216,4.926902,-1.031758,1.253926,0.012291,0.083333,2.968840,0.005109,-1.058833,1.322256,1.253926,0.000000,0.666667,0.346040,0.011846,2.802323,5.456076,-2.507852,0.007663,0.097959,2.572227,-0.018816,0.301792,0.150941,-2.507852,0.0,0.615385,-0.979136,0.007585,2.212536,1.909034,-2.507852,0.641249,0.5625,3.280400,0.004588,1.819316,-0.573947,-1.253926,0.202008,0.2,-0.888167,-0.007832,3.325176,-0.750609,1.253926,0.025169,0.8,0.541774,-0.006570,-0.405945,-0.714724,0.000000,0.00000,0.00,-0.933719,0.014991,-0.314699,1.020862,0.000000,0.005777,0.363636,-0.103108,0.067591,-0.971717,-0.170720,2.507852,0.000000,0.142857,0.454989,0.001850,-0.106857,0.434313,0.000000,0.029042,0.2,-0.556044,-0.004992,-0.381780,-3.006734,3.761778,0.000000,0.684211
3,-0.509467,0.001025,4.462285,-0.594983,1.253926,0.001432,0.000000,2.459552,-0.001424,-0.745463,0.440834,1.253926,0.000000,0.333333,-0.577491,0.001222,2.070250,3.180322,-2.507852,0.016630,0.085714,2.988006,-0.005440,-0.365539,0.912978,-2.507852,0.0,0.615385,-1.716865,0.007866,3.012139,2.359689,-2.507852,0.704073,0.5000,3.136900,-0.008593,1.523889,-3.454502,0.000000,0.057188,0.1,0.102343,-0.005147,3.173354,0.875259,1.253926,0.008606,0.6,0.034836,-0.004085,0.439157,-0.365501,1.253926,0.00000,0.00,-1.028384,-0.001696,0.351662,0.376300,0.000000,0.000000,0.272727,0.257751,0.070887,-1.142335,-0.664724,2.507852,0.000000,0.000000,0.259462,0.002944,0.247345,2.463368,1.253926,0.000000,0.2,0.044105,0.003798,0.328966,-5.640728,6.269630,0.000000,0.736842
4,-0.487675,0.001289,4.494819,-0.234604,0.000000,0.007390,0.083333,2.251643,-0.003517,-0.727909,2.661229,1.253926,0.000000,0.333333,-0.724819,-0.000493,1.827282,2.054305,-1.253926,0.000000,0.077551,3.197019,-0.001012,-0.433167,-1.364231,-2.507852,0.0,0.615385,-2.038091,-0.006820,3.405365,2.007972,-2.507852,0.759332,0.2500,3.168658,-0.005589,1.575199,-1.655047,0.000000,0.044948,0.1,-0.102086,0.000014,3.207281,0.540272,1.253926,0.000000,0.6,0

### Create Feature Interactions

### Two way 

These interactions will be created between categorical features 

In [36]:
categorical_cols = list(set(train.columns) - set(binary_cols))
categorical_cols = [c for c in categorical_cols if c not in ('y','ID')]

In [37]:
train_test = pd.concat([train.loc[:,categorical_cols], test.loc[:,categorical_cols]],axis=0)

In [38]:
train_test.shape

(8417, 8)

In [39]:
import itertools

In [40]:
def encode(charcode):
    r = 0
    ln = len(str(charcode))
    for i in range(ln):
        r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
    return r

In [41]:
 for comb in itertools.combinations(categorical_cols, 2):
        feat = comb[0] + "_" + comb[1]
        train_test[feat] = train_test[comb[0]] + train_test[comb[1]]
        train_test[feat] = train_test[feat].apply(encode)
        print('Analyzing Columns:', feat)

Analyzing Columns: X6_X3
Analyzing Columns: X6_X0
Analyzing Columns: X6_X5
Analyzing Columns: X6_X2
Analyzing Columns: X6_X8
Analyzing Columns: X6_X1
Analyzing Columns: X6_X4
Analyzing Columns: X3_X0
Analyzing Columns: X3_X5
Analyzing Columns: X3_X2
Analyzing Columns: X3_X8
Analyzing Columns: X3_X1
Analyzing Columns: X3_X4
Analyzing Columns: X0_X5
Analyzing Columns: X0_X2
Analyzing Columns: X0_X8
Analyzing Columns: X0_X1
Analyzing Columns: X0_X4
Analyzing Columns: X5_X2
Analyzing Columns: X5_X8
Analyzing Columns: X5_X1
Analyzing Columns: X5_X4
Analyzing Columns: X2_X8
Analyzing Columns: X2_X1
Analyzing Columns: X2_X4
Analyzing Columns: X8_X1
Analyzing Columns: X8_X4
Analyzing Columns: X1_X4


### Three way

In [42]:
for comb in itertools.combinations(categorical_cols, 3):
        feat = comb[0] + "_" + comb[1] + "_" + comb[2]
        train_test[feat] = train_test[comb[0]] + train_test[comb[1]]
        train_test[feat] = train_test[feat].apply(encode)
        print('Analyzing Columns:', feat)

Analyzing Columns: X6_X3_X0
Analyzing Columns: X6_X3_X5
Analyzing Columns: X6_X3_X2
Analyzing Columns: X6_X3_X8
Analyzing Columns: X6_X3_X1
Analyzing Columns: X6_X3_X4
Analyzing Columns: X6_X0_X5
Analyzing Columns: X6_X0_X2
Analyzing Columns: X6_X0_X8
Analyzing Columns: X6_X0_X1
Analyzing Columns: X6_X0_X4
Analyzing Columns: X6_X5_X2
Analyzing Columns: X6_X5_X8
Analyzing Columns: X6_X5_X1
Analyzing Columns: X6_X5_X4
Analyzing Columns: X6_X2_X8
Analyzing Columns: X6_X2_X1
Analyzing Columns: X6_X2_X4
Analyzing Columns: X6_X8_X1
Analyzing Columns: X6_X8_X4
Analyzing Columns: X6_X1_X4
Analyzing Columns: X3_X0_X5
Analyzing Columns: X3_X0_X2
Analyzing Columns: X3_X0_X8
Analyzing Columns: X3_X0_X1
Analyzing Columns: X3_X0_X4
Analyzing Columns: X3_X5_X2
Analyzing Columns: X3_X5_X8
Analyzing Columns: X3_X5_X1
Analyzing Columns: X3_X5_X4
Analyzing Columns: X3_X2_X8
Analyzing Columns: X3_X2_X1
Analyzing Columns: X3_X2_X4
Analyzing Columns: X3_X8_X1
Analyzing Columns: X3_X8_X4
Analyzing Columns: X

In [43]:
train_test.head()

,X6,X3,X0,X5,X2,X8,X1,X4,X6_X3,X6_X0,X6_X5,X6_X2,X6_X8,X6_X1,X6_X4,X3_X0,X3_X5,X3_X2,X3_X8,X3_X1,X3_X4,X0_X5,X0_X2,X0_X8,X0_X1,X0_X4,X5_X2,X5_X8,X5_X1,X5_X4,X2_X8,X2_X1,X2_X4,X8_X1,X8_X4,X1_X4,X6_X3_X0,X6_X3_X5,X6_X3_X2,X6_X3_X8,X6_X3_X1,X6_X3_X4,X6_X0_X5,X6_X0_X2,X6_X0_X8,X6_X0_X1,X6_X0_X4,X6_X5_X2,X6_X5_X8,X6_X5_X1,X6_X5_X4,X6_X2_X8,X6_X2_X1,X6_X2_X4,X6_X8_X1,X6_X8_X4,X6_X1_X4,X3_X0_X5,X3_X0_X2,X3_X0_X8,X3_X0_X1,X3_X0_X4,X3_X5_X2,X3_X5_X8,X3_X5_X1,X3_X5_X4,X3_X2_X8,X3_X2_X1,X3_X2_X4,X3_X8_X1,X3_X8_X4,X3_X1_X4,X0_X5_X2,X0_X5_X8,X0_X5_X1,X0_X5_X4,X0_X2_X8,X0_X2_X1,X0_X2_X4,X0_X8_X1,X0_X8_X4,X0_X1_X4,X5_X2_X8,X5_X2_X1,X5_X2_X4,X5_X8_X1,X5_X8_X4,X5_X1_X4,X2_X8_X1,X2_X8_X4,X2_X1_X4,X8_X1_X4
0,j,a,k,u,at,o,v,d,1125,1135,1145,29302,1139,1146,1128,901,911,23218,905,912,894,1171,29978,1165,1172,1154,36738,1425,1432,1414,23707,23714,23696,1276,1258,1440,1125,1125,1125,1125,1125,1125,1135,1135,1135,1135,1135,1145,1145,1145,1145,29302,29302,29302,1139,1139,1146,901,901,901,901,901,911,911,911,911,23218,23218,23218,905,905,912,1171,1171,1171,1171,29978,29978,29978,1165,1165,1172,36738,36738,36738,1425,1425,1432,23707,23707,23714,1276
1,l,e,k,y,av,o,t,d,1181,1187,1201,30656,1191,1196,1180,1005,1019,25924,1009,1014,998,1175,29980,1165,1170,1154,39444,1529,1534,1518,23759,23764,23748,1274,1258,1388,1181,1181,1181,1181,1181,1181,1187,1187,1187,1187,1187,1201,1201,1201,1201,30656,30656,30656,1191,1191,1196,1005,1005,1005,1005,1005,1019,1019,1019,1019,25924,25924,25924,1009,1009,1014,1175,1175,1175,1175,29980,29980,29980,1165,1165,1170,39444,39444,39444,1529,1529,1534,23759,23759,23764,1274
2,j,c,az,x,n,x,w,d,1127,29308,1148,1138,1148,1147,1128,24576,966,956,966,965,946,23872,23862,23872,23871,23852,1502,1512,1511,1492,1252,1251,1232,1511,1492,1466,1127,1127,1127,1127,1127,1127,29308,29308,29308,29308,29308,1148,1148,1148,1148,1138,1138,1138,1148,1148,1147,24576,24576,24576,24576,24576,966,966,966,966,956,956,956,966,966,965,23872,23872,23872,23872,23862,23862,23862,23872,23872,23871,1502,1502,1502,1512,1512,1511,1252,1252,1251,1511
3,l,f,az,x,n,e,t,d,1182,30660,1200,1190,1181,1196,1180,26604,1044,1034,1025,1040,1024,23872,23862,23853,23868,23852,1502,1493,1508,1492,1233,1248,1232,1014,998,1388,1182,1182,1182,1182,1182,1182,30660,30660,30660,30660,30660,1200,1200,1200,1200,1190,1190,1190,1181,1181,1196,26604,26604,26604,26604,26604,1044,1044,1044,1044,1034,1034,1034,1025,1025,1040,23872,23872,23872,23872,23862,23862,23862,23853,23853,23868,1502,1502,1502,1493,1493,1508,1233,1233,1248,1014
4,d,f,az,h,n,n,v,d,974,25252,976,982,982,990,972,26604,1028,1034,1034,1042,1024,23856,23862,23862,23870,23852,1086,1086,1094,1076,1242,1250,1232,1250,1232,1440,974,974,974,974,974,974,25252,25252,25252,25252,25252,976,976,976,976,982,982,982,982,982,990,26604,26604,26604,26604,26604,1028,1028,1028,1028,1034,1034,1034,1034,1034,1042,23856,23856,23856,23856,23862,23862,23862,23862,23862,23870,1086,1086,1086,1086,1086,1094,1242,1242,1250,1250


### tsne Features

In [44]:
from sklearn.manifold import TSNE


In [45]:
tsne_data = pd.concat([train.loc[:,binary_cols], test.loc[:,binary_cols]])

In [46]:
tsne = TSNE(random_state=2016, perplexity=5, verbose=2)
x = tsne.fit_transform(tsne_data)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 8417
[t-SNE] Computed conditional probabilities for sample 2000 / 8417
[t-SNE] Computed conditional probabilities for sample 3000 / 8417
[t-SNE] Computed conditional probabilities for sample 4000 / 8417
[t-SNE] Computed conditional probabilities for sample 5000 / 8417
[t-SNE] Computed conditional probabilities for sample 6000 / 8417
[t-SNE] Computed conditional probabilities for sample 7000 / 8417
[t-SNE] Computed conditional probabilities for sample 8000 / 8417
[t-SNE] Computed conditional probabilities for sample 8417 / 8417
[t-SNE] Mean sigma: 0.000000
[t-SNE] Iteration 25: error = 2.3300867, gradient norm = 0.0074764
[t-SNE] Iteration 50: error = 2.3138182, gradient norm = 0.0083942
[t-SNE] Iteration 75: error = 2.1781912, gradient norm = 0.0031887
[t-SNE] Iteration 100: error = 2.1355109, gradient norm = 0.0027764
[t-SNE] KL divergence afte

In [47]:
tsne_x = pd.DataFrame(x)
tsne_x.columns = ['tsne1','tsne2']
tsne_x.head()

,tsne1,tsne2
0,6.533732,2.690721
1,-0.706651,-8.082999
2,-1.773410,7.982768
3,-5.251811,-1.866709
4,0.988853,-0.991528


In [48]:
ntrain = train.shape[0]
tsne_train = tsne_x[:ntrain]
tsne_test = tsne_x[ntrain:]

In [49]:
print (tsne_train.shape)
print (tsne_test.shape)

(4208, 2)
(4209, 2)


Now doing t-SNE with selected features from scripus script <br />
https://www.kaggle.com/scirpus/four-blob-tsne

In [50]:
features = ['X118',
            'X127',
            'X47',
            'X315',
            'X311',
            'X179',
            'X314',
            'X261']

In [51]:
tsne_data = pd.concat([train.loc[:,features], test.loc[:,features]])

In [52]:
tsne = TSNE(random_state=2016, perplexity=5, verbose=2)
x = tsne.fit_transform(tsne_data)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 8417
[t-SNE] Computed conditional probabilities for sample 2000 / 8417
[t-SNE] Computed conditional probabilities for sample 3000 / 8417
[t-SNE] Computed conditional probabilities for sample 4000 / 8417
[t-SNE] Computed conditional probabilities for sample 5000 / 8417
[t-SNE] Computed conditional probabilities for sample 6000 / 8417
[t-SNE] Computed conditional probabilities for sample 7000 / 8417
[t-SNE] Computed conditional probabilities for sample 8000 / 8417
[t-SNE] Computed conditional probabilities for sample 8417 / 8417
[t-SNE] Mean sigma: 0.000000
[t-SNE] Iteration 25: error = 0.8188373, gradient norm = 0.0060056
[t-SNE] Iteration 50: error = 0.8119267, gradient norm = 0.0068789
[t-SNE] Iteration 75: error = 0.7962213, gradient norm = 0.0045939
[t-SNE] Iteration 100: error = 0.7929724, gradient norm = 0.0060327
[t-SNE] KL divergence afte

In [53]:
tsne_y = pd.DataFrame(x)
tsne_y.columns = ['tsnef1','tsnef2']
tsne_y.head()

,tsnef1,tsnef2
0,0.748878,9.363999
1,-1.924557,0.084488
2,6.545215,-2.877857
3,6.545233,-2.877865
4,6.545235,-2.877865


In [54]:
tsney_train = tsne_y[:ntrain]
tsney_test = tsne_y[ntrain:]

### row sum

In [55]:
train['row_sum'] = train.loc[:,binary_cols].sum(axis=1)
test['row_sum'] = test.loc[:,binary_cols].sum(axis=1)

### is_duplicated row

#### train data

In [56]:
feature_columns = [x for x in train.columns if 'X' in x]

In [59]:
label_columns = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [60]:
dup_train = train[train.duplicated(subset=feature_columns, keep=False)].sort_values(by=label_columns)
a = dup_train.groupby(feature_columns).apply(lambda x: list(x.ID))

In [61]:
train_index = []
for k,v in a.iteritems():
    train_index.append(v)

In [63]:
dup_train_index = pd.DataFrame({'thelists':train_index})
dup_train_index['code'] = np.arange(217)

In [64]:
final_index = []
for index, row in dup_train_index.iterrows():
    for x in train['ID']:
        if x in row['thelists']:
            final_index.append(row['code'])
    
    

In [65]:
train_index_new = pd.DataFrame({'indexes':[y for x in train_index for y in x],'zcode':final_index})

In [66]:
train['is_duplicate'] = train.ID.map(train_index_new.set_index('indexes')['zcode'].to_dict())

In [67]:
train['is_duplicate'].fillna(-1, inplace=True)

#### test data

In [68]:
dup_test = test[test.duplicated(subset=feature_columns, keep=False)].sort_values(by=label_columns)
a = dup_test.groupby(feature_columns).apply(lambda x: list(x.ID))
test_index = []
for k,v in a.iteritems():
    test_index.append(v)

In [69]:
dup_test_index = pd.DataFrame({'thelists':test_index})
dup_test_index['code'] = np.arange(217)
final_index = []
for index, row in dup_test_index.iterrows():
    for x in test['ID']:
        if x in row['thelists']:
            final_index.append(row['code'])

In [70]:
test_index_new = pd.DataFrame({'indexes':[y for x in test_index for y in x], 'zcode':final_index})
test['is_duplicate'] = test.ID.map(test_index_new.set_index('indexes')['zcode'].to_dict())

In [71]:
test['is_duplicate'].fillna(-1, inplace=True)

### load train_sum_features and test_sum_feature

In [72]:
train_sum_feature = pd.read_csv('train_sum_feature.csv')
test_sum_feature = pd.read_csv('test_sum_feature.csv')

In [73]:
train['sum_row_23'] = train_sum_feature['sum_row_2_to_3']
test['sum_row_23'] = test_sum_feature['sum_row_2_to_3']

In [74]:
del train_sum_feature
del test_sum_feature

### mean X0

In [76]:
meanX0 = train.groupby('X0')['y'].agg('mean')
mean_magic = meanX0['y'].mean()

In [78]:
train['meanX0'] = train['X0'].map(meanX0.to_dict())
test['meanX0'] = test['X0'].map(meanX0.to_dict())
test['meanX0'].fillna(mean_magic, inplace=True)

In [79]:
del meanX0

In [80]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X234,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X291,X292,X294,X295,X296,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385,row_sum,is_duplicate,sum_row_23,meanX0
0,0,130.81,k,v,at,a,d,u,j,o,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,52,-1.0,5,99.491818
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,52,-1.0,5,99.491818
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0

### Loading features from R

In [81]:
logpca_train = pd.read_csv('logpca_train_v2.csv') 
logpca_test = pd.read_csv('logpca_test_v2.csv')

In [82]:
dist_train = pd.read_csv('dist_train_v2.csv')
dist_test = pd.read_csv('dist_test_v2.csv')

### combining all features

In [91]:
# exporting all data to R for cbind, having trouble with pandasb

# print (train.shape)
# print (logpca_train.shape)
# print (dist_train.shape)
# print (tsne_train.shape)
# print (tsney_train.shape)
# print (dim_train.shape)

(4208, 370)
(4208, 20)
(4208, 2)
(4208, 2)
(4208, 2)
(4208, 84)


In [99]:
train_embed = train_test[:ntrain]
test_embed = train_test[ntrain:]

In [100]:
print (train_embed.shape)
print (test_embed.shape)

(4208, 92)
(4209, 92)


In [106]:
cols_select = [c for c in train_embed.columns if len(str(c)) > 2 ]

In [107]:
train_embed = train_embed[cols_select]
test_embed = test_embed[cols_select]

In [109]:
train.to_csv("new_features/train_v2.csv",index=False)
test.to_csv("new_features/test_v2.csv",index=False)
dim_train.to_csv("new_features/dim_train_v2.csv",index=False)
dim_test.to_csv("new_features/dim_test_v2.csv", index=False)
tsne_train.to_csv("new_features/tsne_train_v2.csv",index=False)
tsne_test.to_csv("new_features/tsne_test_v2.csv",index=False)
tsney_train.to_csv("new_features/tsney_train_v2.csv",index=False)
tsney_test.to_csv("new_features/tsney_test_v2.csv",index=False)
train_embed.to_csv("new_features/train_embed_v2.csv",index=False)
test_embed.to_csv("new_features/test_embed_v2.csv",index=False)

In [110]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X234,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X291,X292,X294,X295,X296,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385,row_sum,is_duplicate,sum_row_23,meanX0
0,0,130.81,k,v,at,a,d,u,j,o,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,52,-1.0,5,99.491818
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,52,-1.0,5,99.491818
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0

### read merged files from R

In [135]:
del train,test,train_embed,test_embed,tsney_test,tsney_train,tsne_data,tsne_train,tsne_test

In [119]:
full_train = pd.read_csv("new_features/full_train_v2.csv")
full_test = pd.read_csv("new_features/full_test_v2.csv")

In [120]:
print (full_train.shape)
print (full_test.shape)

(4208, 200)
(4209, 199)


In [137]:
def get_sklearn_stack_data(model,train,col,label,test):
    ID = []
    R2_Score = []
    RMSE = []
    train = train.reset_index(drop=True)
    kf = KFold(n_splits=5,shuffle=False)
    i=0
    for train_index, test_index in kf.split(train):
        
        print("Training "+str(i+1)+' Fold')
        X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
        y_train, y_test = label.iloc[train_index],label.iloc[test_index]
        
        model.fit(X_train[col],y_train)
        pred = model.predict(X_test[col])
        
        X_test['label'] = list(y_test)
        X_test['y'] = pred
        
        r2 = r2_score(y_test,pred)
        rmse = MSE(y_test,pred)**0.5
        
        print('R2 Scored of Fold '+str(i+1)+' is '+str(r2))
        R2_Score.append(r2)
        RMSE.append(rmse)
        
        print('RMSE of Fold '+str(i+1)+' is '+str(rmse))
        ID.append(X_test['ID'])
        
        if i==0:
            Final = X_test
        else:
            Final = Final.append(X_test,ignore_index=True)
        i+=1
    print('Start Training')
    model.fit(train[col],label)
    Final_pred = model.predict(test[col])
    Final_pred = pd.DataFrame({'ID':test['ID'],'y':Final_pred})
    print('Calculating In-Bag R2 Score')
    print(r2_score(label, model.predict(train[col])))
    print('Calculating Out-Bag R2 Score')
    print(np.mean(R2_Score))
    print('Calculating In-Bag RMSE')
    print(MSE(label, model.predict(train[col]))**0.5)
    print('Calculating Out-Bag RMSE')
    print(np.mean(RMSE))
    return Final,Final_pred

In [139]:
cols = full_test.columns

In [242]:
## Input 1: GBDT

gb1 = GradientBoostingRegressor(n_estimators=1000,max_features=0.5,learning_rate=0.005,max_depth=6)
gb1_train,gb1_test = get_sklearn_stack_data(gb1,full_train,cols,full_train['y'],full_test)

Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 1 is 0.526242411593
RMSE of Fold 1 is 8.50538878396
Training 2 Fold
R2 Scored of Fold 2 is 0.546424814543
RMSE of Fold 2 is 8.83918903533
Training 3 Fold
R2 Scored of Fold 3 is 0.578557495238
RMSE of Fold 3 is 8.01659905698
Training 4 Fold
R2 Scored of Fold 4 is 0.512055250453
RMSE of Fold 4 is 8.79803219447
Training 5 Fold
R2 Scored of Fold 5 is 0.540006222377
RMSE of Fold 5 is 7.7913993233
Start Training
Calculating In-Bag R2 Score
0.796828874971
Calculating Out-Bag R2 Score
0.540657238841
Calculating In-Bag RMSE
5.59945725598
Calculating Out-Bag RMSE
8.39012167881


In [243]:
## Input 2: Random Forest

rf1 = RandomForestRegressor(n_estimators=1000, max_features=0.3,max_depth=6,min_samples_split=7)
rf_train,rf1_test = get_sklearn_stack_data(rf1, full_train, cols, full_train['y'], full_test)

Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 1 is 0.602262043491
RMSE of Fold 1 is 7.79317806577
Training 2 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 2 is 0.543082387301
RMSE of Fold 2 is 8.87169755675
Training 3 Fold


...

R2 Scored of Fold 3 is 0.5995003583
RMSE of Fold 3 is 7.81487547157
Training 4 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 4 is 0.539838670337
RMSE of Fold 4 is 8.54388279142
Training 5 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 5 is 0.629221175948
RMSE of Fold 5 is 6.99514862248
Start Training
Calculating In-Bag R2 Score
0.655412479531
Calculating Out-Bag R2 Score
0.582780927075
Calculating In-Bag RMSE
7.29230313412
Calculating Out-Bag RMSE
8.0037565016


In [244]:
## Input 3: KNN Regressor

knn1 = KNeighborsRegressor(n_neighbors=7)
kn_train,knn_test = get_sklearn_stack_data(knn1, full_train, cols, full_train['y'], full_test)


Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 1 is 0.183737906214
RMSE of Fold 1 is 11.1642844947
Training 2 Fold
R2 Scored of Fold 2 is 0.218894981385
RMSE of Fold 2 is 11.5995860904
Training 3 Fold
R2 Scored of Fold 3 is 0.202274959171
RMSE of Fold 3 is 11.0292913327
Training 4 Fold
R2 Scored of Fold 4 is 0.244316849518
RMSE of Fold 4 is 10.9488899771
Training 5 Fold
R2 Scored of Fold 5 is 0.207555075521
RMSE of Fold 5 is 10.2264295038
Start Training
Calculating In-Bag R2 Score
0.4704761292
Calculating Out-Bag R2 Score
0.211355954362
Calculating In-Bag RMSE
9.0397769026
Calculating Out-Bag RMSE
10.9936962797


In [245]:
## Input 4: Lasso

las1 = Lasso(alpha=0.003,random_state=42)
las1_train,las1_test = get_sklearn_stack_data(las1,full_train,cols,full_train['y'],full_test)

Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

R2 Scored of Fold 1 is 0.586460450331
RMSE of Fold 1 is 7.94647654232
Training 2 Fold
R2 Scored of Fold 2 is 0.5442207879
RMSE of Fold 2 is 8.8606388406
Training 3 Fold
R2 Scored of Fold 3 is 0.600881761184
RMSE of Fold 3 is 7.80138630026
Training 4 Fold
R2 Scored of Fold 4 is 0.54072511658
RMSE of Fold 4 is 8.53564943576
Training 5 Fold
R2 Scored of Fold 5 is 0.630089456533
RMSE of Fold 5 is 6.98695328964
Start Training
Calculating In-Bag R2 Score
0.613689779773
Calculating Out-Bag R2 Score
0.580475514506
Calculating In-Bag RMSE
7.72116867459
Calculating Out-Bag RMSE
8.02622088172


In [246]:
## Input 5: Extratrees

from sklearn.ensemble import ExtraTreesRegressor
ext1 = ExtraTreesRegressor(n_estimators=1000, max_depth=6, max_features=0.3,min_samples_split=7)
ext1_train,ext1_test = get_sklearn_stack_data(ext1,full_train,cols,full_train['y'],full_test)

Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 1 is 0.608609313329
RMSE of Fold 1 is 7.73074456997
Training 2 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 2 is 0.539761071941
RMSE of Fold 2 is 8.90388317624
Training 3 Fold


...

R2 Scored of Fold 3 is 0.593913312254
RMSE of Fold 3 is 7.86919617999
Training 4 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 4 is 0.537080271104
RMSE of Fold 4 is 8.56945233129
Training 5 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 5 is 0.627722663789
RMSE of Fold 5 is 7.00926990558
Start Training
Calculating In-Bag R2 Score
0.623019047788
Calculating Out-Bag R2 Score
0.581417326483
Calculating In-Bag RMSE
7.62736701906
Calculating Out-Bag RMSE
8.01650923261


In [154]:
def get_lgb_stack_data(params,rounds,train,col,label,test):
    ID = []
    train = train.reset_index(drop=True)
    kf = KFold(n_splits=5,shuffle=False)
    i=0
    R2_Score = []
    RMSE = []
    for train_index, test_index in kf.split(train):
        print("Training "+str(i+1)+' Fold')
        X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
        y_train, y_test = label.iloc[train_index],label.iloc[test_index]
        train_lgb=lgb.Dataset(X_train[col],y_train)
        model = lgb.train(params,train_lgb,num_boost_round=rounds)
        pred = model.predict(X_test[col])
        X_test['label'] = list(y_test)
        X_test['y'] = pred
        r2 = r2_score(y_test,pred)
        rmse = MSE(y_test,pred)**0.5
        print('R2 Scored of Fold '+str(i+1)+' is '+str(r2))
        R2_Score.append(r2)
        RMSE.append(rmse)
        print('RMSE of Fold '+str(i+1)+' is '+str(rmse))
        ID.append(X_test['ID'])
        if i==0:
            Final = X_test
        else:
            Final = Final.append(X_test,ignore_index=True)
        i+=1
    lgb_train_ = lgb.Dataset(train[col],label)
    print('Start Training')
    model_ = lgb.train(params,lgb_train_,num_boost_round=rounds)
    Final_pred = model_.predict(test[col])
    Final_pred = pd.DataFrame({'ID':test['ID'],'y':Final_pred})
    print('Calculating In-Bag R2 Score')
    print(r2_score(label, model.predict(train[col])))
    print('Calculating Out-Bag R2 Score')
    print(np.mean(R2_Score))
    print('Calculating In-Bag RMSE')
    print(MSE(label, model.predict(train[col]))**0.5)
    print('Calculating Out-Bag RMSE')
    print(np.mean(RMSE))
    return Final,Final_pred

In [247]:
# light gbm
params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'num_iterations': 500,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 3,
            'num_rounds': 800
        }
lgb_train, lgb_test = get_lgb_stack_data(params,800,full_train,cols,full_train['y'],full_test)

Training 1 Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 Scored of Fold 1 is 0.61267628589
RMSE of Fold 1 is 7.69047428502
Training 2 Fold
R2 Scored of Fold 2 is 0.544224504679
RMSE of Fold 2 is 8.86060271225
Training 3 Fold
R2 Scored of Fold 3 is 0.603913035664
RMSE of Fold 3 is 7.77170434926
Training 4 Fold
R2 Scored of Fold 4 is 0.538316521029
RMSE of Fold 4 is 8.55800210943
Training 5 Fold
R2 Scored of Fold 5 is 0.63622905705
RMSE of Fold 5 is 6.92872760251
Start Training
Calculating In-Bag R2 Score
0.625236187857
Calculating Out-Bag R2 Score
0.587071880862
Calculating In-Bag RMSE
7.60490450614
Calculating Out-Bag RMSE
7.96190221169


### Make a lgb submission

In [239]:
sub_lgb = pd.DataFrame({'ID':test.ID, 'y':np.exp(lgb_test['y'] + 1)})
sub_lgb.to_csv("new_features/sub_8thJuly_lightgbm.csv", index=False)

In [156]:
def get_xgb_stack_data(params, rounds, train, col, label, test):
    ID = []
    train = train.reset_index(drop=True)
    kf = KFold(n_splits=5, shuffle=False)
    i = 0
    R2_Score = []
    RMSE = []
    for train_index, test_index in kf.split(train):
        print ('Training '+str(i + 1)+'Fold')
        
        X_train, X_test = train.iloc[train_index], train.iloc[test_index]
        y_train, y_test = label.iloc[train_index], label.iloc[test_index]
        
        dtrain = xgb.DMatrix(X_train[col],y_train)
        dtest = xgb.DMatrix(X_test[col])
        model = xgb.train(params, dtrain, rounds)
        pred = model.predict(dtest)
        
        X_test['label'] = list(y_test)
        X_test['y'] = pred
        
        r2 = r2_score(y_test, pred)
        rmse = np.sqrt(MSE(y_test, pred))
        
        print ('R2 scored of Fold' +str(i+1)+'is'+str(r2))
        R2_Score.append(r2)
        RMSE.append(rmse)
        print('RMSE of Fold '+str(i+1)+' is '+str(rmse))
        ID.append(X_test['ID'])
        
        if i ==0:
            Final = X_test
        else: 
            Final = Final.append(X_test, ignore_index = True)
            
        i+=1
    dtrain_ = xgb.DMatrix(train[col],label)
    dtest_ = xgb.DMatrix(test[col])
    
    print('Start Training')
    model_ = xgb.train(params,dtrain_,num_boost_round=rounds)
    Final_pred = model_.predict(dtest_)
    Final_pred = pd.DataFrame({'ID':test['ID'],'y':Final_pred})
    
    print('Calculating In-Bag R2 Score')
    print(r2_score(dtrain_.get_label(), model.predict(dtrain_)))
    
    print('Calculating Out-Bag R2 Score')
    print(np.mean(R2_Score))
    
    print('Calculating In-Bag RMSE')
    print(MSE(dtrain_.get_label(), model.predict(dtrain_))**0.5)
    
    print('Calculating Out-Bag RMSE')
    print(np.mean(RMSE))
    
    return Final,Final_pred
        
        


In [248]:
# xgb

y_mean = np.median(full_train['y'])

xg_params = {
    'eta': 0.005,
    'max_depth': 4,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'subsample':0.95,
    'colsample_bytree':0.4,
    'base_score': y_mean # base prediction = mean(target)
}

xg_train, xg_test = get_xgb_stack_data(xg_params, 1000, full_train, cols, full_train['y'], full_test)

Training 1Fold


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R2 scored of Fold1is0.561543884227
RMSE of Fold 1 is 8.1823706126
Training 2Fold
R2 scored of Fold2is0.542459259285
RMSE of Fold 2 is 8.8777449487
Training 3Fold
R2 scored of Fold3is0.600254598338
RMSE of Fold 3 is 7.80751333054
Training 4Fold
R2 scored of Fold4is0.533804049748
RMSE of Fold 4 is 8.5997231552
Training 5Fold
R2 scored of Fold5is0.630927609275
RMSE of Fold 5 is 6.97903319224
Start Training
Calculating In-Bag R2 Score
0.682794944257
Calculating Out-Bag R2 Score
0.573797880175
Calculating In-Bag RMSE
6.99656702056
Calculating Out-Bag RMSE
8.08927704786


In [227]:
# level 1 models

stack_train = gb1_train.loc[:,['label','predicted']]
stack_train.columns = [['label','gbdt']]
stack_train['forest'] = rf_train.loc[:,'predicted']
stack_train['knn'] = kn_train.loc[:,['predicted']]
stack_train['lasso'] = las1_train.loc[:,['predicted']]
stack_train['trees'] = ext1_train.loc[:,['predicted']]
stack_train['light'] = lgb_train.loc[:,['predicted']]
stack_train['xgb'] = xg_train.loc[:,['predicted']]

In [255]:
# level 1 models

stack_test = gb1_test.loc[:,['ID','y']]
stack_test.columns = [['ID','gbdt']]
stack_test['forest'] = rf1_test.loc[:,'y']
stack_test['knn'] = knn_test.loc[:,['y']]
stack_test['lasso'] = las1_test.loc[:,['y']]
stack_test['trees'] = ext1_test.loc[:,['y']]
stack_test['light'] = lgb_test.loc[:,['y']]
stack_test['xgb'] = xg_test.loc[:,['y']]
del stack_test['ID']

### Before, stacked model we'll do model median averaging

In [256]:
stack_test['mean_col'] = stack_test.mean(axis=1)
stack_test.head()

,gbdt,forest,knn,lasso,trees,light,xgb,mean_col
0,78.167271,78.867653,84.181429,78.769530,78.539089,79.316609,79.678406,79.645712
1,96.429743,95.115240,98.032857,95.383656,94.487129,94.609426,95.087029,95.592154
2,79.241523,77.633699,102.918571,79.568068,78.202583,79.086946,79.396507,82.292557
3,78.749937,78.187098,87.261429,78.426366,78.214386,78.890575,79.365944,79.870819
4,113.172341,112.357873,106.670000,114.052322,111.959189,112.228437,113.064468,111.929233


In [257]:
sub_ave = pd.DataFrame({'ID':test.ID,'y':stack_test.mean_col})
sub_ave.to_csv("new_features/model_averaging_8thjuly.csv",index=False) # 0.5534

### meta model

In [231]:
y_mean = np.median(stack_train['label'])

col = list(stack_test.columns)

xg_params = {
    'eta': 0.005,
    'max_depth': 6,
    'objective': 'reg:linear',
    'subsample':0.95,
    'colsample_bytree':0.5,
    'base_score': y_mean # base prediction = mean(target)
}

dtrain = xgb.DMatrix(stack_train[col], stack_train['label'])
dtest = xgb.DMatrix(stack_test[col])

In [232]:
model = xgb.train(params,dtrain,num_boost_round =5000,verbose_eval=30)
pred_1 = model.predict(dtest)

In [236]:
sub = pd.DataFrame({'ID':test['ID'],'y':np.exp(pred_1 + 1)})
sub.to_csv("new_features/stacked_models_data8june.csv",index=False) # -178

In [241]:
sub.head()

,ID,y
0,1,213.429184
1,2,256.097076
2,3,213.016083
3,4,215.275818
4,5,307.467743
